# Pythonic SysML Elements

This notebook walks through the approach to making SysML elements act Pythonically while in the PyMBE environment in order to support more intuitive codes that inspect and analyze SysML models.

Note that the Markdown headers for code snippets are set up to be translated into docstrings for documentation of the code eventually.

In [ ]:
import inspect
import os
import traceback
import logging
import pymbe.api as pm
from pymbe.model import Element
from pymbe.model import ListOfNamedItems
from pymbe.model import Model

from pymbe.model_modification import new_element_ownership_pattern, build_from_feature_pattern

import json
from dataclasses import dataclass, field
from typing import Any, Dict, List, Set, Tuple, Union, Collection
from collections import defaultdict

from uuid import uuid4

## Raw Data to Process into Elements

PyMBE expects to encounter model data in the form serialized in the REST API. This means that model elements are first seen as Python dictionaries, with references to other elements as IDs. The entries in the dictionary are driven by using the reflective Ecore model of KerML and SysML v2 to determine which attributes to include.

In [ ]:
empty_model = pm.Model(elements={})
empty_model

Fill in a quick namespace and Package so elements we explore have a place to live.

In [ ]:
package_model_namespace_data = {
    'aliasIds': [],
    'isImpliedIncluded': False,
    '@type': "Namespace",
    '@id': str(uuid4()),
    'ownedRelationship': []
}
package_model_data = {
    'name': "Example Builder Model",
    'isLibraryElement': False,
    'filterCondition': [],
    'ownedElement': [],
    'owner': {},
    '@type': "Package",
    '@id': str(uuid4()),
    'ownedRelationship': []
}
new_ns = Element.new(data=package_model_namespace_data,model=empty_model)
new_package = Element.new(data=package_model_data,model=empty_model)
new_element_ownership_pattern(
    owner=new_ns, ele=new_package, model=empty_model, member_kind="OwningMembership"
)

An example of this data set is generated for a PartDefinition and a PartUsage below. Additional tags like "@id" and "@type" get generated when a new element is made.

In [ ]:
partdefinition_data = empty_model.metamodel.pre_made_dicts["PartDefinition"]
partusage_data = empty_model.metamodel.pre_made_dicts["PartUsage"]
partusage_data

In [ ]:
partdefinition_data.update({"@type": "PartDefinition", "declaredName": "Demo Unit"})
partdefinition_data.update({"@id": str(uuid4())})
partusage_data.update({"@type": "PartUsage", "declaredName": "Demo Component"})
partusage_data.update({"@id": str(uuid4())})
partusage_data

In [ ]:
partdefinition_data

To demonstration how relationships are made, an example FeatureMembership is needed.

In [ ]:
fm_data = empty_model.metamodel.pre_made_dicts["FeatureMembership"]

In [ ]:
fm_data

Linking this to the other elements requires reference to their IDs.

In [ ]:
fm_data.update({"source": [{"@id": partdefinition_data["@id"]}],
                "target": [{"@id": partusage_data["@id"]}],
                "owningRelatedElement": {"@id": partdefinition_data["@id"]},
               "@id": str(uuid4()),
                "@type": "FeatureMembership"})
fm_data

In [ ]:
element_functions = inspect.getmembers(Element, inspect.isfunction)
element_functions

## Pythonic Model elements

The methods and the class developed here are intended to support the Python representation and manipulation of system model data. The partial methods describe below set up various fields in objects of the Element class for use in downstream or user codes involving this library.

There are a variety of features developed here to make the interaction with system model data more Pythonic.

The getattr magic is used to map a simple field name to a getitem looking for a key against the data (or derived cache) dictionary of the Python element. The key will return data from non-derived attributes directly, or will call helper functions to calculate the derived attributes.

In [ ]:
print(inspect.getsource(element_functions[5][1]))

When the Python object is created from the dictionary, it becomes possible to access known fields directly.

In [ ]:
partdefinition_ele = Element.new(data=partdefinition_data, model=empty_model)
partdefinition_ele

### Accessing fields Pythonically

The next thing to add to the Element class are magics to match the meta-attributes of loaded elements to properties on Python objects. That is done by adding to the getattr and getitem magics.

This should focus on the metafields with expected literal values in the JSON.

In [ ]:
partdefinition_ele.declaredName

Derived attributes can also be computed. The below will be empty since there are no direct Features (and library is not loaded to show the Features of the Part object).

In [ ]:
partdefinition_ele.feature

### Navigating relationships Pythonically

The next Pythonic task for our model elements is the ability to traverse SysML v2 relationships Pythonically, either in the forward (through) or reverse direction with simple property accessors.

In [ ]:
partusage_ele = Element.new(data=partusage_data, model=empty_model)
fm_ele = Element.new(data=fm_data, model=empty_model)

empty_model._add_relationship(fm_ele)

In [ ]:
model_functions = inspect.getmembers(Model, inspect.isfunction)

In addition to laying in the attributes on Python objects, the PyMBE framework will create two kinds of attributes, a "through(RelationshipType)" and a "reverse(RelationshipType)", that can be used to navigate across standard SysML relationships like Subclassification, Subsetting, etc.

In [ ]:
print(inspect.getsource(model_functions[7][1]))

In [ ]:
partdefinition_ele.throughFeatureMembership

In [ ]:
partusage_ele.reverseFeatureMembership

The add relationship function drives the model class to dereference the IDs from the element and make these Python-style attributes.

### List of Items Accessible by Name

For convenience, a class is introduced that allows for collections of objects to be referenced by name rather than by unique identifier. This requires that names in a given list are unique. This is applied to the model-level object for directly owned elements (those elements with no other model element as an owner).

In [ ]:
empty_model.ownedElement["Demo Unit"]